In [ ]:
import numpy as np
import plyfile
import pyvista as pv
import matplotlib.pyplot as plt

1. Load point cloud

In [ ]:
filename = '/home/lyy/workspace/Instruct-Replacement/datasets/scannet/scans/scene0000_00/scene0000_00_vh_clean_2.ply' 
with open(filename, 'rb') as f:
    plydata = plyfile.PlyData.read(f)
    num_verts = plydata['vertex'].count
    vertices = np.zeros(shape=[num_verts, 6], dtype=np.float32)
    vertices[:,0] = plydata['vertex'].data['x']
    vertices[:,1] = plydata['vertex'].data['y']
    vertices[:,2] = plydata['vertex'].data['z']
    vertices[:,3] = plydata['vertex'].data['red']
    vertices[:,4] = plydata['vertex'].data['green']
    vertices[:,5] = plydata['vertex'].data['blue']

2. Create a pyvista point cloud object and generate mesh

In [ ]:
# Create a pyvista point cloud object
cloud = pv.PolyData(vertices[:, :3])

# Generate the mesh
mesh = cloud.delaunay_2d()
colors = vertices[:, 3:].astype(np.uint8)

3. Plot the mesh

In [ ]:
# Plot the mesh
plotter = pv.Plotter()
plotter.add_mesh(mesh, scalars=colors, rgb=True, preference='point')
plotter.show()
# Save the plot graph in pdf format
plotter.save_graphic("test1.pdf")

The following is optional for save mesh

In [ ]:
# mesh does not have color information
mesh.save('mesh.vtk', texture=)
